In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# === Setup dendPLRNN on Kaggle ===
!git clone --depth 1 https://github.com/DurstewitzLab/dendPLRNN.git /kaggle/working/dendPLRNN

%cd /kaggle/working/dendPLRNN/BPTT_TF

# Install missing dependencies
!pip install tensorboardX

# Ensure Python can find the modules
import sys
sys.path.append("/kaggle/working/dendPLRNN")
sys.path.append("/kaggle/working/dendPLRNN/BPTT_TF")

print("Setup complete — open and run BPTT_TF/example.ipynb")


fatal: destination path '/kaggle/working/dendPLRNN' already exists and is not an empty directory.
/kaggle/working/dendPLRNN/BPTT_TF
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.9 MB/s eta 0:00:00
Setup complete — open and run BPTT_TF/example.ipynb


In [3]:
import os

ROOT = "/kaggle/working/dendPLRNN"

def tree(dir_path, prefix=""):
    files = sorted(os.listdir(dir_path))
    for i, f in enumerate(files):
        path = os.path.join(dir_path, f)
        connector = "└── " if i == len(files)-1 else "├── "
        print(prefix + connector + f)
        if os.path.isdir(path):
            extension = "    " if i == len(files)-1 else "│   "
            tree(path, prefix + extension)

print("Project structure:\n")
tree(ROOT)


Project structure:

├── .git
│   ├── HEAD
│   ├── branches
│   ├── config
│   ├── description
│   ├── hooks
│   │   ├── applypatch-msg.sample
│   │   ├── commit-msg.sample
│   │   ├── fsmonitor-watchman.sample
│   │   ├── post-update.sample
│   │   ├── pre-applypatch.sample
│   │   ├── pre-commit.sample
│   │   ├── pre-merge-commit.sample
│   │   ├── pre-push.sample
│   │   ├── pre-rebase.sample
│   │   ├── pre-receive.sample
│   │   ├── prepare-commit-msg.sample
│   │   ├── push-to-checkout.sample
│   │   └── update.sample
│   ├── index
│   ├── info
│   │   └── exclude
│   ├── logs
│   │   ├── HEAD
│   │   └── refs
│   │       ├── heads
│   │       │   └── main
│   │       └── remotes
│   │           └── origin
│   │               └── HEAD
│   ├── objects
│   │   ├── info
│   │   └── pack
│   │       ├── pack-1942f751a4058b4f6e9d40182c2cd324e746fae5.idx
│   │       └── pack-1942f751a4058b4f6e9d40182c2cd324e746fae5.pack
│   ├── packed-refs
│   ├── refs
│   │   ├── heads
│   │   │   └──

In [4]:
# Automatically modify training parameters
!sed -i 's/maxepoch = [0-9]\+/maxepoch = 200/' ubermain.py
!sed -i 's/patience = [0-9]\+/patience = 15/' ubermain.py


In [6]:
# Show only the lines that contain maxepoch and patience
!grep -nE "maxepoch|patience" ubermain.py


In [8]:
!sed -n '1,320p' ubermain.py


from multitasking import *


def ubermain(n_runs):
    """
    Specify the argument choices you want to be tested here in list format:
    e.g. args.append(Argument('dim_z', [5, 6], add_to_name_as='z'))
    will test for dimensions 5 and 6 and save experiments under z5 and z6.
    Possible Arguments can be found in main.py
    
    When using GPU for training (i.e. Argument 'use_gpu 1')  it is generally
    not necessary to specify device ids, tasks will be distributed automatically.
    """
    args = []
    args.append(Argument('experiment', ['ubermain_example']))
    args.append(Argument('data_path', ['examples/Lorenz63/lorenz.npy']))
    args.append(Argument('dim_z', [15]))
    args.append(Argument('fix_obs_model', [1]))
    args.append(Argument('layer_norm', [1]))
    args.append(Argument('learn_z0', [1]))
    args.append(Argument('n_epochs', [1000]))
    args.append(Argument('n_interleave', [15, 30], add_to_name_as="tau"))
    args.append(Argument('seq_len', [200]))
    args.appe

In [9]:
# Search for parameters controlling epochs / training loops
!grep -nEi "epoch|epochs|iter|max|train|patien" ubermain.py

# Also search calls to train() function
!grep -nEi "train\(" ubermain.py

# Also search in the underlying training function
!grep -nEi "epoch|epochs|patien" ../bptt/*.py


11:    When using GPU for training (i.e. Argument 'use_gpu 1')  it is generally
21:    args.append(Argument('n_epochs', [1000]))
grep: ../bptt/*.py: No such file or directory


In [10]:
!grep -nEi "patien" ubermain.py


In [11]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG

# Modify epoch count
!sed -i "s/Argument('n_epochs', \\[[0-9]\+\\])/Argument('n_epochs', [200])/" ubermain.py

# Verify change
!grep -nEi "n_epochs" ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
23:    args.append(Argument('n_epochs', [200]))


In [12]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
!python ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
Traceback (most recent call last):
  File "/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/ubermain.py", line 42, in <module>
    args = ubermain(n_runs)
           ^^^^^^^^^^^^^^^^
  File "/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/ubermain.py", line 15, in ubermain
    args.append(Argument('experiment', ['ECG']))
                ^^^^^^^^
NameError: name 'Argument' is not defined


In [13]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG

# Add the required import if missing
!sed -i "1s/^/from multitasking import Argument\n/" ubermain.py

# Verify the import was added
!head -n 20 ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
from multitasking import Argument
from multitasking import *


def ubermain(n_runs):
    """
    Specify the argument choices you want to be tested here in list format:
    e.g. args.append(Argument('dim_z', [5, 6], add_to_name_as='z'))
    will test for dimensions 5 and 6 and save experiments under z5 and z6.
    Possible Arguments can be found in main.py
    
    When using GPU for training (i.e. Argument 'use_gpu 1')  it is generally
    not necessary to specify device ids, tasks will be distributed automatically.
    """
    args = []
    args.append(Argument('experiment', ['ECG']))
    args.append(Argument('data_path', ['Experiments/Table1/ECG/data/ECG_train.npy']))
    args.append(Argument('use_gpu', [0])) # may wanna use gpu here
    args.append(Argument('dim_z', [30], add_to_name_as="M"))
    args.append(Argument('n_bases', [50], add_to_name_as="B"))


In [14]:
!python ubermain.py


Traceback (most recent call last):
  File "/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/ubermain.py", line 1, in <module>
    from multitasking import Argument
ImportError: cannot import name 'Argument' from 'multitasking' (/usr/local/lib/python3.11/dist-packages/multitasking/__init__.py)


In [15]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG

# Ensure Python can see the BPTT_TF folder
import sys
sys.path.append("/kaggle/working/dendPLRNN/BPTT_TF")

# Modify ubermain.py to import the correct Argument class
!sed -i "s/from multitasking import Argument/from bptt.multitasking import Argument/" ubermain.py

# If the line does not exist yet, we add it at the top safely
!sed -i "1s/^/from bptt.multitasking import Argument\n/" ubermain.py

# Show first 15 lines to confirm
!head -n 15 ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
from bptt.multitasking import Argument
from bptt.multitasking import Argument
from multitasking import *


def ubermain(n_runs):
    """
    Specify the argument choices you want to be tested here in list format:
    e.g. args.append(Argument('dim_z', [5, 6], add_to_name_as='z'))
    will test for dimensions 5 and 6 and save experiments under z5 and z6.
    Possible Arguments can be found in main.py
    
    When using GPU for training (i.e. Argument 'use_gpu 1')  it is generally
    not necessary to specify device ids, tasks will be distributed automatically.
    """


In [16]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG

# Remove any accidental "from multitasking import *" line
!sed -i "/from multitasking import/d" ubermain.py

# Remove duplicate Argument imports, keep only one
!sed -i "0,/from bptt.multitasking import Argument/{/from bptt.multitasking import Argument/{n;d;}}" ubermain.py

# Show first 15 lines to verify
!head -n 15 ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
from bptt.multitasking import Argument


def ubermain(n_runs):
    """
    Specify the argument choices you want to be tested here in list format:
    e.g. args.append(Argument('dim_z', [5, 6], add_to_name_as='z'))
    will test for dimensions 5 and 6 and save experiments under z5 and z6.
    Possible Arguments can be found in main.py
    
    When using GPU for training (i.e. Argument 'use_gpu 1')  it is generally
    not necessary to specify device ids, tasks will be distributed automatically.
    """
    args = []
    args.append(Argument('experiment', ['ECG']))


In [17]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
!python ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
Traceback (most recent call last):
  File "/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/ubermain.py", line 1, in <module>
    from bptt.multitasking import Argument
ModuleNotFoundError: No module named 'bptt'


In [18]:
import sys
sys.path.append("/kaggle/working/dendPLRNN")
sys.path.append("/kaggle/working/dendPLRNN/BPTT_TF")

print("PYTHONPATH updated.")


PYTHONPATH updated.


In [19]:
import bptt
print("bptt module successfully loaded.")


bptt module successfully loaded.


In [20]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
!python ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
Traceback (most recent call last):
  File "/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/ubermain.py", line 1, in <module>
    from bptt.multitasking import Argument
ModuleNotFoundError: No module named 'bptt'


In [21]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
!PYTHONPATH="/kaggle/working/dendPLRNN/BPTT_TF:/kaggle/working/dendPLRNN" python ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
Traceback (most recent call last):
  File "/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/ubermain.py", line 1, in <module>
    from bptt.multitasking import Argument
ModuleNotFoundError: No module named 'bptt.multitasking'


In [22]:
import sys
sys.path.insert(0, "/kaggle/working/dendPLRNN/BPTT_TF")
sys.path.insert(0, "/kaggle/working/dendPLRNN")
print("Local project PATH added.")


Local project PATH added.


In [23]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG

# Remove incorrect old imports
!sed -i "/from bptt.multitasking import/d" ubermain.py
!sed -i "/from multitasking import/d" ubermain.py

# Insert the correct import at the top
!sed -i "1s/^/from multitasking import Argument\n/" ubermain.py

# Show first lines to verify
!head -n 10 ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
from multitasking import Argument


def ubermain(n_runs):
    """
    Specify the argument choices you want to be tested here in list format:
    e.g. args.append(Argument('dim_z', [5, 6], add_to_name_as='z'))
    will test for dimensions 5 and 6 and save experiments under z5 and z6.
    Possible Arguments can be found in main.py
    


In [24]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
!PYTHONPATH="/kaggle/working/dendPLRNN/BPTT_TF:/kaggle/working/dendPLRNN" python ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
Traceback (most recent call last):
  File "/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/ubermain.py", line 43, in <module>
    run_settings(*create_tasks_from_arguments(args, n_proc_per_gpu, n_cpu))
    ^^^^^^^^^^^^
NameError: name 'run_settings' is not defined


In [25]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG

# Remove incorrect imports just to avoid duplicates
!sed -i "/from multitasking import/d" ubermain.py

# Insert the correct import at the top
!sed -i "1s/^/from multitasking import Argument, run_settings, create_tasks_from_arguments\n/" ubermain.py

# Verify beginning of file
!head -n 15 ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
from multitasking import Argument, run_settings, create_tasks_from_arguments


def ubermain(n_runs):
    """
    Specify the argument choices you want to be tested here in list format:
    e.g. args.append(Argument('dim_z', [5, 6], add_to_name_as='z'))
    will test for dimensions 5 and 6 and save experiments under z5 and z6.
    Possible Arguments can be found in main.py
    
    When using GPU for training (i.e. Argument 'use_gpu 1')  it is generally
    not necessary to specify device ids, tasks will be distributed automatically.
    """
    args = []
    args.append(Argument('experiment', ['ECG']))


In [26]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
!PYTHONPATH="/kaggle/working/dendPLRNN/BPTT_TF:/kaggle/working/dendPLRNN" python ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
python3: can't open file '/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/main.py': [Errno 2] No such file or directory
python3: can't open file '/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/main.py': [Errno 2] No such file or directory
python3: can't open file '/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/main.py': [Errno 2] No such file or directory
python3: can't open file '/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/main.py': [Errno 2] No such file or directory
python3: can't open file '/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/main.py': [Errno 2] No such file or directory
python3: can't open file '/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/main.py': [Errno 2] No such file or directory
python3: can't open file '/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/main.py': [Errno 2] No such file or directory
python3: can't open file '/kaggle/workin